In [10]:
import pandas as pd
import torch
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [4]:
df.iloc[:, 0]

,id
0,842302
1,842517
2,84300903
3,84348301
4,84358402
...,...
564,926424
565,926682
566,926954
567,927241


In [5]:
df = df.iloc[:,1:32]

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop('diagnosis', axis = 1),df['diagnosis'],train_size=0.8,random_state= True)

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)
x_scaled = scaler.fit_transform(X_test)

In [8]:
from sklearn.preprocessing import LabelEncoder
Y_encoded = LabelEncoder().fit_transform(y_train)
y_encoded = LabelEncoder().fit_transform(y_test)

In [11]:
x_train_tensor = torch.from_numpy(X_scaled)
x_test_tensor = torch.from_numpy(x_scaled)
y_train_tensor = torch.from_numpy(Y_encoded)
y_test_tensor = torch.from_numpy(y_encoded)

MODEL

In [19]:
class NN():
  def __init__(self,X):
    self.wt = torch.rand(X.shape[1],1,dtype = torch.float64,requires_grad= True)
    self.bias = torch.zeros(1,dtype = torch.float64,requires_grad= True)
  def forward(self,X):
    Z = torch.matmul(X,self.wt) + self.bias
    return torch.sigmoid(Z)
  def loss_function(self, y_pred, y):
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)
    # Calculate loss
    loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
    return loss



In [31]:
lr_rate = 0.1
epochs = 20

In [32]:
model = NN(x_train_tensor)
for epoch in range(epochs):
  y_pred = model.forward(x_train_tensor)
  loss = model.loss_function(y_pred,y_train_tensor)
  print(f'epoch:{epoch},loss: {loss}')
  loss.backward()
  with torch.no_grad():
   model.wt -= lr_rate + model.wt.grad
   model.bias -= lr_rate + model.bias.grad
  model.wt.grad.zero_()
  model.bias.grad.zero_()
  print(f'epoch:{epoch},loss: {loss}')

epoch:0,loss: 3.6239831532711695
epoch:0,loss: 3.6239831532711695
epoch:1,loss: 1.4630164959746068
epoch:1,loss: 1.4630164959746068
epoch:2,loss: 1.487375908645515
epoch:2,loss: 1.487375908645515
epoch:3,loss: 0.7262926904266995
epoch:3,loss: 0.7262926904266995
epoch:4,loss: 0.804721459418977
epoch:4,loss: 0.804721459418977
epoch:5,loss: 0.7347267807339251
epoch:5,loss: 0.7347267807339251
epoch:6,loss: 0.8143110129548472
epoch:6,loss: 0.8143110129548472
epoch:7,loss: 0.7572503971527133
epoch:7,loss: 0.7572503971527133
epoch:8,loss: 0.8143481830620103
epoch:8,loss: 0.8143481830620103
epoch:9,loss: 0.7846131072900988
epoch:9,loss: 0.7846131072900988
epoch:10,loss: 0.8170406028578708
epoch:10,loss: 0.8170406028578708
epoch:11,loss: 0.8080073318047458
epoch:11,loss: 0.8080073318047458
epoch:12,loss: 0.8248713254091664
epoch:12,loss: 0.8248713254091664
epoch:13,loss: 0.8257552868427951
epoch:13,loss: 0.8257552868427951
epoch:14,loss: 0.8353338667975083
epoch:14,loss: 0.8353338667975083
epoc

In [34]:
with torch.no_grad():
 y_pred = model.forward(x_train_tensor)
 y_pred = (y_pred > 0.5).float()

In [41]:
accuracy = (y_pred == y_test_tensor).float().mean()
print(accuracy)

tensor(0.5801)
